# Hugging Faceで公開されている大規模言語モデルをSageMakerにデプロイ


* 対象モデル
  
  cyberagent/open-calm-7b
  
  https://huggingface.co/cyberagent/open-calm-7b

* HuggingFace Text Generation Inference Containers

  https://huggingface.co/blog/sagemaker-huggingface-llm

  https://aws.amazon.com/jp/blogs/machine-learning/announcing-the-launch-of-new-hugging-face-llm-inference-containers-on-amazon-sagemaker/


### SageMakerライブラリーのインストール

In [ ]:
%pip install sagemaker --upgrade


---

### パラメーターを指定

In [ ]:
model_id = 'cyberagent/open-calm-7b'
instance_type = 'ml.g5.12xlarge'
gpus = '4'


### インポート

In [ ]:
import sagemaker
import boto3


### IAMロールの取得

In [ ]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role_name = 'AmazonSageMaker-ExecutionRole-20230617T201891' # Role name with `AmazonSageMakerFullAccess` policy attached
	role = iam.get_role(RoleName=role_name)['Role']['Arn']


### SageMakerへデプロイ

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.huggingface import get_huggingface_llm_image_uri

image_uri = get_huggingface_llm_image_uri(
  backend='huggingface', # or lmi
  # region=region
)

# Hub model configuration <https://huggingface.co/models>
hub = {
  'HF_MODEL_ID': model_id, # model_id from hf.co/models
  'HF_TASK':'text-generation',          # NLP task you want to use for predictions
  # 'HF_MODEL_QUANTIZE':'bitsandbytes',
  'SM_NUM_GPUS': gpus,
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
  env=hub,                            # configuration for loading model from Hub
  role=role,                          # IAM role with permissions to create an endpoint
  image_uri=image_uri
)


# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=600,
)


### 推論

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

data = {
  'inputs': 'AIによって私達の暮らしは、',
  'parameters': {
    'max_new_tokens': 64,
    'do_sample': True,
    'temperature': 0.7,
    'top_p': 0.9,
    'repetition_penalty': 1.05,
    'pad_token_id': tokenizer.pad_token_id,
  }
}

# request
result = predictor.predict(data)

result


### エンドポイントの削除

In [ ]:
predictor.delete_endpoint(delete_endpoint_config=False)
predictor.delete_model()
